# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ancud,-41.8697,-73.8203,17.68,49,39,2.24,CL,1743884426
1,1,tsiombe,-25.3000,45.4833,23.55,85,44,4.19,MG,1743884427
2,2,teguise,29.0605,-13.5640,18.12,64,20,3.60,ES,1743884229
3,3,adamstown,-25.0660,-130.1015,24.20,78,100,6.43,PN,1743884429
4,4,ierapetra,35.0100,25.7353,14.52,73,43,1.79,GR,1743884431


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Humidity"]
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] >= 25) &\
                            (city_data_df['Max Temp'] <= 30)]
ideal_cities_df = ideal_cities_df[(ideal_cities_df['Humidity'] >= 40) &\
                                (ideal_cities_df['Humidity'] <= 60)]


# Drop any rows with null values
ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,barentu,15.1139,37.5928,26.25,45,80,4.46,ER,1743884489
197,197,katiola,8.1333,-5.1000,27.95,59,97,1.79,CI,1743884680
315,315,karratha,-20.7377,116.8463,27.67,56,0,3.10,AU,1743884839
376,376,karachi,24.9056,67.0822,25.90,57,0,3.09,PK,1743884715
391,391,ad darb,17.7229,42.2526,29.64,46,95,2.82,SA,1743884933
394,394,sechura,-5.5569,-80.8222,29.54,55,67,4.30,PE,1743884936
489,489,bluefield,37.2698,-81.2223,27.79,42,40,3.60,US,1743885060
550,550,san luis de la loma,17.2706,-100.8939,29.57,57,6,5.07,MX,1743885147
557,557,beledweyne,4.7358,45.2036,28.09,55,21,8.37,SO,1743885155


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
52,barentu,ER,15.1139,37.5928,45,None
197,katiola,CI,8.1333,-5.1000,59,None
315,karratha,AU,-20.7377,116.8463,56,None
376,karachi,PK,24.9056,67.0822,57,None
391,ad darb,SA,17.7229,42.2526,46,None
394,sechura,PE,-5.5569,-80.8222,55,None
489,bluefield,US,37.2698,-81.2223,42,None
550,san luis de la loma,MX,17.2706,-100.8939,57,None
557,beledweyne,SO,4.7358,45.2036,55,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "type": "accommodation.hotel",
    "limit": 1,
    "radius": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
barentu - nearest hotel: No hotel found
katiola - nearest hotel: Hôtel Amitie
karratha - nearest hotel: Karratha International Hotel
karachi - nearest hotel: Quetta Agha Hotel
ad darb - nearest hotel: No hotel found
sechura - nearest hotel: Maximus Inn
bluefield - nearest hotel: Econo Lodge Near Bluefield College
san luis de la loma - nearest hotel: No hotel found
beledweyne - nearest hotel: Hotel Medina


,City,Country,Lat,Lng,Humidity,Hotel Name
52,barentu,ER,15.1139,37.5928,45,No hotel found
197,katiola,CI,8.1333,-5.1000,59,Hôtel Amitie
315,karratha,AU,-20.7377,116.8463,56,Karratha International Hotel
376,karachi,PK,24.9056,67.0822,57,Quetta Agha Hotel
391,ad darb,SA,17.7229,42.2526,46,No hotel found
394,sechura,PE,-5.5569,-80.8222,55,Maximus Inn
489,bluefield,US,37.2698,-81.2223,42,Econo Lodge Near Bluefield College
550,san luis de la loma,MX,17.2706,-100.8939,57,No hotel found
557,beledweyne,SO,4.7358,45.2036,55,Hotel Medina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)